In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-156307
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-156307


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name = "project1Cluster"
try:
    target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Same cluster name compute exists. Reusing the same")
except: 
    print("Compute with same cluster name doesn't exist and hence creating NEW")
    config_compute = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes = 5)
    target = ComputeTarget.create(ws, cluster_name, config_compute)

target.wait_for_completion(show_output=True)
print(target.get_status().serialize())


Compute with same cluster name doesn't exist and hence creating NEW
InProgress.....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2021-08-30T15:09:13.519000+00:00', 'errors': None, 'creationTime': '2021-08-30T15:09:13.163852+00:00', 'modifiedTime': '2021-08-30T15:09:38.731649+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 5, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [4]:
print(2+3)

5


In [25]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C' : choice(0.001,0.01,0.1,1,10,50,100,150,200,300,400, 500,1000),
    '--max_iter' : choice(50,100,150,200,300)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
script_folder = './training'
os.makedirs(script_folder, exist_ok=True)
import shutil
shutil.copy('./train.py', script_folder)
est = SKLearn(
    source_directory= script_folder,
    compute_target=target,
    entry_script="train.py"
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=policy,
    estimator=est,
    max_total_runs=15,
    max_concurrent_runs=3

)

In [26]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
run_hyp = exp.submit(hyperdrive_config)
RunDetails(run_hyp).show()

run_hyp.wait_for_completion(show_output = True)
assert(run_hyp.get_status() =="Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_ad433613-a86c-4a2f-81bd-d141e613d0d6
Web View: https://ml.azure.com/runs/HD_ad433613-a86c-4a2f-81bd-d141e613d0d6?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-156307/workspaces/quick-starts-ws-156307&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-08-30T16:28:33.341256][API][INFO]Experiment created<END>\n""<START>[2021-08-30T16:28:33.853479][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space<END>\n""<START>[2021-08-30T16:28:34.024687][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_ad433613-a86c-4a2f-81bd-d141e613d0d6
Web View: https://ml.azure.com/runs/HD_ad433613-a86c-4a2f-81bd-d141e613d0d6?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-156307/workspaces/quick-starts-ws-156307&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



KeyError: 'log_files'

In [ ]:
#Running for Hyperdrive for AUC

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C' : choice(0.001,0.01,0.1,1,10,50,100,150,200,300,400, 500,1000),
    '--max_iter' : choice(50,100,150,200,300)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
script_folder = './training'
os.makedirs(script_folder, exist_ok=True)
import shutil
shutil.copy('./train.py', script_folder)
est = SKLearn(
    source_directory= script_folder,
    compute_target=target,
    entry_script="train.py"
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config_2 = HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_name='AUC Weighted',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=policy,
    estimator=est,
    max_total_runs=15,
    max_concurrent_runs=3

)

run_hyp_2 = exp.submit(hyperdrive_config)
RunDetails(run_hyp_2).show()

run_hyp_2.wait_for_completion(show_output = True)


In [29]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = run_hyp.get_best_run_by_primary_metric()
print("Best Run File names:")
print(best_run.get_file_names())
print("Best Run Details:")
print(best_run.get_details())
print("Best Run Metrics")
print(best_run.get_metrics())

Best Run File names:
['azureml-logs/55_azureml-execution-tvmps_63193fe43ef0cf3688fc53f7f774ce857de480464b88fd34dc69e51d9c60f7bc_d.txt', 'azureml-logs/65_job_prep-tvmps_63193fe43ef0cf3688fc53f7f774ce857de480464b88fd34dc69e51d9c60f7bc_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_63193fe43ef0cf3688fc53f7f774ce857de480464b88fd34dc69e51d9c60f7bc_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/105_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']
Best Run Details:
{'runId': 'HD_ad433613-a86c-4a2f-81bd-d141e613d0d6_7', 'target': 'project1Cluster', 'status': 'Completed', 'startTimeUtc': '2021-08-30T16:34:46.183357Z', 'endTimeUtc': '2021-08-30T16:36:09.635059Z', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': 'de2c63f0-4c2b-477b-9a3b-3279ebab589c', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azure

In [30]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [31]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [33]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=4)

In [34]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output = False)
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_115d9386-8552-4390-83de-7baa8d783928,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [36]:
# Retrieve and save your best automl model.

### YOUR CO##
run_automl = automl_run.get_best_child()
run_automl.get_details()

{'runId': 'AutoML_115d9386-8552-4390-83de-7baa8d783928_38',
 'status': 'Completed',
 'startTimeUtc': '2021-08-30T17:48:49.77465Z',
 'endTimeUtc': '2021-08-30T17:49:45.573289Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'AUC_weighted\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-project\',\'compute_target\':\'local\',\'subscription_id\':\'b968fb36-f06a-4c76-a15f-afab68ae7667\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_115d9386-8552-4390-83de-7baa8d783928_38","experiment_name":null,"workspace_name":"quick-starts-ws-156307","subscription_id":"b968fb36-f06a-4c76-a15f-afab68ae7667","resource_group_name":"aml-